In [1]:
!git clone https://github.com/GoGoDuck912/Self-Correction-Human-Parsing.git
%cd Self-Correction-Human-Parsing

import os
os.makedirs("/kaggle/working/human-parsing", exist_ok=True)
os.makedirs("/kaggle/working/cloth-mask", exist_ok=True)
os.makedirs("/kaggle/working/json_keypoints", exist_ok=True)

!pip install inplace-abn
!pip install -r requirements.txt

print("Deleting outdated 'modules' folder...")
!rm -rf ./modules
print("Fixing import statement...")
!sed -i 's/from modules import InPlaceABNSync/from inplace_abn import InPlaceABNSync/' ./networks/AugmentCE2P.py
print("Fixing activation function name...")
!sed -i "s/activation='none'/activation='identity'/g" ./networks/AugmentCE2P.py
print("\nAll fixes applied successfully!")

!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install opencv-python matplotlib
!pip install tfjs-graph-converter

!git clone https://github.com/rwightman/posenet-pytorch
%cd posenet-pytorch

fatal: destination path 'Self-Correction-Human-Parsing' already exists and is not an empty directory.
/kaggle/working/Self-Correction-Human-Parsing
  Using cached opencv-python-4.4.0.46.tar.gz (88.9 MB)
  Installing build dependencies ...   error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
Deleting outdated 'modules' folder...
Fixing import statement...
Fixing activation function name...

All fixes applied successfully!
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-uebptise
  Running command git clone --filter

In [2]:
def parsing_map(image_dir, output_dir="/kaggle/working/human-parsing"):
    """
    Runs the human parsing script from the correct directory.
    """
    # This command assumes you've already run the setup cell that clones the repo
    # and installs dependencies.
    %cd /kaggle/working/Self-Correction-Human-Parsing
    
    !python3 simple_extractor.py \
        --dataset lip \
        --model-restore /kaggle/input/lip-modellll/exp-schp-201908261155-lip.pth \
        --input-dir {image_dir} \
        --output-dir {output_dir}

In [3]:
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamPredictor
import os

def cloth_mask(cloth_path, output_dir="/kaggle/working/cloth-mask"):
    """
    Creates a segmentation mask for a single cloth image.
    """
    # (SAM model loading logic is the same...)
    sam_checkpoint = "/kaggle/input/segment-anything/pytorch/vit-b/1/model.pth"
    model_type = "vit_b"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    sam = sam_model_registry[model_type](checkpoint=sam_checkpoint).to(device)
    predictor = SamPredictor(sam)

    image = cv2.imread(cloth_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    predictor.set_image(image)
    
    h, w = image.shape[:2]
    input_point = np.array([[w // 2, h // 2]])
    input_label = np.array([1])
    
    masks, scores, _ = predictor.predict(
        point_coords=input_point, point_labels=input_label, multimask_output=True)
        
    best_mask = (masks[np.argmax(scores)] * 255).astype(np.uint8)
    
    # <-- FIX: Create a dynamic output name.
    os.makedirs(output_dir, exist_ok=True)
    base_name = os.path.basename(cloth_path)
    file_name_without_ext = os.path.splitext(base_name)[0]
    output_filename = f"{file_name_without_ext}_mask.png"
    output_path = os.path.join(output_dir, output_filename)
    
    cv2.imwrite(output_path, best_mask)
    print(f"Saved cloth mask to {output_path}")


In [4]:
import os
import json
import torch
import numpy as np
import posenet
# --- FIX: Added the missing import for load_model ---
from posenet.models.model_factory import load_model

def keypoints(image_path, output_dir="/kaggle/working/json_keypoints"):
    """
    Generates pose keypoints for a single image and saves them to a JSON file.
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # This line will now work correctly
    net = load_model(101).cuda()
    output_stride = net.output_stride
    
    input_image, _, _ = posenet.read_imgfile(
        image_path, scale_factor=1.0, output_stride=output_stride)

    with torch.no_grad():
        input_image = torch.Tensor(input_image).cuda()
        heatmaps, offsets, fwd, bwd = net(input_image)
        pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multiple_poses(
            heatmaps.squeeze(0), offsets.squeeze(0), fwd.squeeze(0), bwd.squeeze(0),
            output_stride=output_stride, max_pose_detections=1, min_pose_score=0.25)
    
    poses = keypoint_coords.astype(np.int32)
    if not len(poses) > 0:
        print(f"Warning: No pose detected for {image_path}")
        return None

    pose = poses[0]
    
    indices = [0, (5, 6), 6, 8, 10, 5, 7, 9, 12, 14, 16, 11, 13, 15, 2, 1, 4, 3]
    openpose = []
    for ix in indices:
        if ix == (5, 6):
            openpose.append([int((pose[5][1] + pose[6][1]) / 2), int((pose[5][0] + pose[6][0]) / 2), 1])
        else:
            openpose.append([int(pose[ix][1]), int(pose[ix][0]), 1])

    coords = [float(item) for sublist in openpose for item in sublist]
    data = {"people": [{"pose_keypoints_2d": coords}]}
    
    base_name = os.path.basename(image_path)
    file_name_without_ext = os.path.splitext(base_name)[0]
    output_filename = f"{file_name_without_ext}_keypoints.json"
    output_path = os.path.join(output_dir, output_filename)
    
    with open(output_path, 'w') as f:
        json.dump(data, f)
    
    return output_path

In [5]:
import cv2
import json
import numpy as np
import os

def open_pose(image_path, keypoints_path, output_dir="/kaggle/working/openpose_image"):
    """
    Draws a pose skeleton on a blank canvas using keypoints from a JSON file.
    Args:
        image_path (str): Path to the original image to get canvas dimensions.
        keypoints_path (str): Path to the JSON file containing the pose keypoints.
        output_dir (str): Directory where the output skeleton image will be saved.
    """
    # --- Load image using the function argument ---
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not load image from {image_path}")
        return
        
    canvas_height, canvas_width = img.shape[:2]
    canvas = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)
    
    # --- Load keypoints using the function argument ---
    with open(keypoints_path) as f:
        keypoints_json = json.load(f)
    
    # Check if keypoints were found
    if not keypoints_json['people']:
        print(f"Error: No people found in keypoints file {keypoints_path}")
        return

    keypoints = np.array(keypoints_json['people'][0]['pose_keypoints_2d']).reshape(-1, 3)
    
    # --- Define pose pairs ---
    POSE_PAIRS = [
        (1, 2), (1, 5), (2, 3), (3, 4), (5, 6),
        (6, 7), (1, 8), (8, 9), (9, 10), (1, 11),
        (11, 12), (12, 13), (0, 1), (0, 14),
        (14, 16), (0, 15), (15, 17)
    ]
    
    # --- Color gradient for limbs (from red to violet) ---
    def get_color(index, total):
        hsv = np.array([[[int(index / total * 180), 255, 255]]], dtype=np.uint8)
        bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)[0][0]
        return tuple(int(c) for c in bgr)
    
    # --- Draw keypoints and colored limbs ---
    for i, (x, y, conf) in enumerate(keypoints):
        if conf > 0.1:
            cv2.circle(canvas, (int(x), int(y)), 5, (255, 255, 255), -1)  # white joint
            # The line that draws the point numbers has been removed from here.
    
    for i, (partA, partB) in enumerate(POSE_PAIRS):
        # This check is needed in case a keypoint is missing from the JSON
        if partA < len(keypoints) and partB < len(keypoints):
            xA, yA, cA = keypoints[partA]
            xB, yB, cB = keypoints[partB]
            if cA > 0.1 and cB > 0.1:
                ptA = (int(xA), int(yA))
                ptB = (int(xB), int(yB))
                color = get_color(i, len(POSE_PAIRS))  # rainbow color
                cv2.line(canvas, ptA, ptB, color, thickness=10)
    
    # --- Save the final skeleton image ---
    os.makedirs(output_dir, exist_ok=True)
    # Create a dynamic output name based on the input image
    base_name = os.path.basename(image_path)
    file_name_without_ext = os.path.splitext(base_name)[0]
    output_filename = f"{file_name_without_ext}_skeleton.png"
    output_path = os.path.join(output_dir, output_filename)
    
    cv2.imwrite(output_path, canvas)
    print(f"Skeleton image saved to: {output_path}")

In [6]:
import os

def pre_processing(cloth_dir, image_dir):
    """
    Runs the full pre-processing pipeline by pairing sorted files from
    the cloth and image directories.
    """
    print("--- Starting Full Pre-processing Pipeline ---")
    
    # Step 1: Run parsing on the entire image directory
    print("\n[1/4] Running human parsing on all images...")
    parsing_map(image_dir) 
    
    # --- FIX: Added '.jpeg' to the list of accepted file extensions ---
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
    cloth_files = sorted([f for f in os.listdir(cloth_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])

    # Check if the directories have the same number of images
    if len(image_files) != len(cloth_files):
        print(f" Warning: Mismatch in file counts. Images: {len(image_files)}, Cloths: {len(cloth_files)}")
        print("Processing only the minimum number of pairs.")
        
    num_pairs = min(len(image_files), len(cloth_files))
    
    # Loop through the number of pairs
    for i in range(num_pairs):
        image_filename = image_files[i]
        cloth_filename = cloth_files[i]
        
        print(f"\n--- Processing Pair {i+1}/{num_pairs}: ---")
        print(f"  Image: {image_filename}")
        print(f"  Cloth: {cloth_filename}")
        
        # Define paths for the current pair
        image_path = os.path.join(image_dir, image_filename)
        cloth_path = os.path.join(cloth_dir, cloth_filename)

        # Step 2: Create cloth mask
        print(f"[2/4] Creating cloth mask for {cloth_filename}...")
        cloth_mask(cloth_path)

        # Step 3: Create keypoints JSON
        print(f"[3/4] Creating keypoints for {image_filename}...")
        keypoints_json_path = keypoints(image_path)
        
        if keypoints_json_path is None:
            print("Skipping skeleton creation due to no keypoints.")
            continue

        # Step 4: Create OpenPose skeleton image
        print(f"[4/4] Creating skeleton image for {image_filename}...")
        open_pose(image_path, keypoints_json_path)
        
    print("\n--- Full Pre-processing Pipeline Finished! ---")

In [7]:
!mkdir /kaggle/working/input_cloth
!mkdir /kaggle/working/input_person

mkdir: cannot create directory ‘/kaggle/working/input_cloth’: File exists
mkdir: cannot create directory ‘/kaggle/working/input_person’: File exists


In [8]:
# !cp /kaggle/input/test-data/00000_00.jpg /kaggle/working/input_person/00891_00.jpg
# !cp /kaggle/input/test-data/00001_00.jpg /kaggle/working/input_cloth/01430_00.jpg



!cp /kaggle/input/test-data2/00891_00.jpg /kaggle/working/input_person/00891_00.jpg
!cp /kaggle/input/test-data2/01430_00.jpg /kaggle/working/input_cloth/01430_00.jpg

In [9]:
# Define your input directories
my_cloth_directory = "/kaggle/working/input_cloth"
my_image_directory = "/kaggle/working/input_person"

# Run the entire pipeline
pre_processing(cloth_dir=my_cloth_directory, image_dir=my_image_directory)

--- Starting Full Pre-processing Pipeline ---

[1/4] Running human parsing on all images...
/kaggle/working/Self-Correction-Human-Parsing
Evaluating total class number 20 with ['Background', 'Hat', 'Hair', 'Glove', 'Sunglasses', 'Upper-clothes', 'Dress', 'Coat', 'Socks', 'Pants', 'Jumpsuits', 'Scarf', 'Skirt', 'Face', 'Left-arm', 'Right-arm', 'Left-leg', 'Right-leg', 'Left-shoe', 'Right-shoe']
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.93it/s]

--- Processing Pair 1/1: ---
  Image: 00891_00.jpg
  Cloth: 01430_00.jpg
[2/4] Creating cloth mask for 01430_00.jpg...
Saved cloth mask to /kaggle/working/cloth-mask/01430_00_mask.png
[3/4] Creating keypoints for 00891_00.jpg...
Cannot find models file ./_models/mobilenet_v1_101.pth, converting from tfjs...
Weights for checkpoint mobilenet_v1_101 are not downloaded. Downloading to /tmp/_posenet_weights ...
[4/4] Creating skeleton image for 00891_00.jpg...
Skeleton image saved to: /kaggle/working/openpose_image/00891

In [ ]:
import cv2
from PIL import Image
import torch

!pip install torchgeometry
!git clone https://github.com/shadow2496/VITON-HD.git

In [25]:
!cp -r /kaggle/input/viton-hd-files/datasets/datasets/test/test /kaggle/working/VITON-HD/datasets
!cp /kaggle/input/viton-hd-files/datasets/datasets/test_pairs.txt /kaggle/working/VITON-HD/test_pairs.txt
!cp /kaggle/input/viton-hd-files/datasets/datasets/test_pairs.txt /kaggle/working/VITON-HD/datasets/test_pairs.txt

!cp -r /kaggle/input/viton-hd-files/checkpoints/checkpoints /kaggle/working/VITON-HD

In [23]:
!cp /kaggle/working/cloth-mask/01430_00_mask.png /kaggle/working/VITON-HD/datasets/test/cloth-mask/01430_00.jpg
!cp /kaggle/working/human-parsing/00891_00.png /kaggle/working/VITON-HD/datasets/test/image-parse/00891_00.png
!cp /kaggle/working/json_keypoints/00891_00_keypoints.json /kaggle/working/VITON-HD/datasets/test/openpose-json/00891_00_keypoints.json
!cp /kaggle/working/openpose_image/00891_00_skeleton.png /kaggle/working/VITON-HD/datasets/test/openpose-img/00891_00_rendered.png

In [ ]:
!cp /kaggle/working/input_person/00891_00.jpg /kaggle/working/VITON-HD/datasets/test/image/00891_00.jpg
!cp /kaggle/working/input_cloth/01430_00.jpg /kaggle/working/VITON-HD/datasets/test/cloth/01430_00.jpg

In [24]:
!cd /kaggle/working/VITON-HD && CUDA_VISIBLE_DEVICES=0 python test.py --name VITON

Namespace(name='VITON', batch_size=1, workers=1, load_height=1024, load_width=768, shuffle=False, dataset_dir='./datasets/', dataset_mode='test', dataset_list='test_pairs.txt', checkpoint_dir='./checkpoints/', save_dir='./results/', display_freq=1, seg_checkpoint='seg_final.pth', gmm_checkpoint='gmm_final.pth', alias_checkpoint='alias_final.pth', semantic_nc=13, init_type='xavier', init_variance=0.02, grid_size=5, norm_G='spectralaliasinstance', ngf=64, num_upsampling_layers='most')
Network [SegGenerator] was created. Total number of parameters: 34.5 million. To see the architecture, do print(network).
Network [ALIASGenerator] was created. Total number of parameters: 100.5 million. To see the architecture, do print(network).
/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5015: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of

In [ ]:
!ls /kaggle/working/VITON-HD

In [31]:
def VITON_HD(user_path, cloth_path, tryon_output_path):

    user_image = Image.open(user_path)
    cloth_image = Image.open(cloth_path)

    my_cloth_directory = "/kaggle/working/input_cloth"
    my_image_directory = "/kaggle/working/input_person"
    
    pre_processing(cloth_dir=my_cloth_directory, image_dir=my_image_directory)

    !cp /kaggle/working/cloth-mask/01430_00_mask.png /kaggle/working/VITON-HD/datasets/test/cloth-mask/01430_00.jpg
    !cp /kaggle/working/human-parsing/00891_00.png /kaggle/working/VITON-HD/datasets/test/image-parse/00891_00.png
    !cp /kaggle/working/json_keypoints/00891_00_keypoints.json /kaggle/working/VITON-HD/datasets/test/openpose-json/00891_00_keypoints.json
    # !cp /kaggle/working/openpose_image/00891_00_skeleton.png /kaggle/working/VITON-HD/datasets/test/openpose-img/00891_00_rendered.png

    !cp /kaggle/working/input_person/00891_00.jpg /kaggle/working/VITON-HD/datasets/test/image/00891_00.jpg
    !cp /kaggle/working/input_cloth/01430_00.jpg /kaggle/working/VITON-HD/datasets/test/cloth/01430_00.jpg

    !cd /kaggle/working/VITON-HD && CUDA_VISIBLE_DEVICES=0 python test.py --name VITON

    result = Image.open("/kaggle/working/VITON-HD/results/VITON/00891_01430_00.jpg")
    result.save(tryon_output_path)

In [32]:
!pip install pyngrok
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import FileResponse
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from PIL import Image
import os

app = FastAPI()

from fastapi.middleware.cors import CORSMiddleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  #
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Make folders to store files
os.makedirs("uploads", exist_ok=True)
os.makedirs("results", exist_ok=True)

@app.post("/vton")
async def vton(cloth: UploadFile = File(...), user: UploadFile = File(...)):
    # Save cloth and user images to disk
    cloth_path = f"uploads/{cloth.filename}"
    user_path = f"uploads/{user.filename}"
    
    with open(cloth_path, "wb") as f:
        f.write(await cloth.read())
    with open(user_path, "wb") as f:
        f.write(await user.read())

    # === Replace this line with your actual VTON code ===
    tryon_output_path = "results/tryon_output.png"
    VITON_HD(user_path, cloth_path, tryon_output_path)

    # Return the result image back to the website
    return FileResponse(tryon_output_path, media_type="image/png")

In [33]:
!ngrok config add-authtoken 30BEI0jrdnlCm5ePSwmrDlYyOmV_7AphnTKcssudeZhUaK5Yp

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
nest_asyncio.apply()
port = 8000
public_url = ngrok.connect(port)
print("🌐 Your public API:", public_url)
uvicorn.run(app, host="0.0.0.0", port=port)